### Gap statistics

In [1]:
%matplotlib inline
import time
import hashlib
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
from gap_statistic import OptimalK

plt.rcParams['figure.figsize'] = 10, 10

C:\Users\sohil\Anaconda3\envs\py36\lib\site-packages\gap_statistic\optimalK.py:15: UserWarning: joblib not installed, will be unavailable as a backend for parallel processing.
  warnings.warn('joblib not installed, will be unavailable as a backend for parallel processing.')


## Read and Preprocess the image

In [2]:
def readImg(imPath):
    import matplotlib.image as mpimg
    import cv2

#     imPath = r"C:\Users\sohil\AnacondaProjects\JobTask\example_output\extract_field%d.jpg"
    # img=mpimg.imread(imPath)
    img = cv2.imread(imPath)
    img = cv2.GaussianBlur(img,(11,11),7)
    img = cv2.pyrMeanShiftFiltering(img, 30, 75)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    ## to grayscale
    # img = Image.open(imPath).convert('LA')
    # img = color.rgb2gray(img)

#     plt.imshow(img)

    # cv2.imshow('Original', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img


### 3D plotting

In [3]:
def filterImg(img):
    # reshape the image
    reshapedImg = np.reshape(img, (img.shape[0]*img.shape[1], 3))
    # filtering: remove black colors
    finalImg = reshapedImg[np.any(reshapedImg, axis=1)]
    # flatten all pixels values under 25
    thresh = 25
    finalImg[(finalImg < thresh).all(axis=1)] = 0
    
    
    removed = (reshapedImg.shape[0] - finalImg.shape[0]) * 100 / reshapedImg.shape[0]
#     print('%d percent of the image removed'%removed)
    return finalImg

### Find outliers

In [4]:
def zScore(finalImg):
    
    from scipy import stats
    import numpy as np
    z = np.abs(stats.zscore(finalImg))
#     print(finalImg.shape)
#     print(finalImg[3000])
#     print(z.shape)
#     print(z[3000])
    return z

### Removing Outliers

In [5]:
def removeOutliers(finalImg, z, threshold):
    threshold = 1.8
#     print(np.where(z > threshold)[0].shape)
#     print('%d, %d before removing'%finalImg.shape)
    finalImg = finalImg[(z < threshold).all(axis=1)]
#     print('%d, %d after removing'%finalImg.shape)
    return finalImg

### Plotting color space

In [6]:
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# ax.scatter(finalImg[:,0], finalImg[:, 1], finalImg[:, 2])

# ax.set_xlabel('Red')
# ax.set_ylabel('Green')
# ax.set_zlabel('Blue')

# plt.show()

### Find Optimal number of clusters

In [7]:
def findOptimalK(finalImg):
    optimalK = OptimalK(parallel_backend='rust')
    
    n_clusters = optimalK(finalImg, cluster_array=np.arange(1,4))
    print('Optimal clusters: %d'% n_clusters)
    return n_clusters
#     optimalK

In [8]:
# %%time
# # X, y = make_blobs(n_samples=int(1e5), n_features=2, centers=3, random_state=25)
# # print('Data shape: ', X.shape)

# n_clusters = optimalK(finalImg, cluster_array=np.arange(1,4))
# print('Optimal clusters: ', n_clusters)

In [9]:
# optimalK.gap_df.head()

In [10]:
# plt.plot(optimalK.gap_df.n_clusters, optimalK.gap_df.gap_value, linewidth=3)
# plt.scatter(optimalK.gap_df[optimalK.gap_df.n_clusters == n_clusters].n_clusters,
#             optimalK.gap_df[optimalK.gap_df.n_clusters == n_clusters].gap_value, s=250, c='r')
# plt.grid(True)
# plt.xlabel('Cluster Count')
# plt.ylabel('Gap Value')
# plt.title('Gap Values by Cluster Count')
# plt.show()

In [11]:

# Now that we have the optimal clusters, n, we build our own KMeans model...
# km = KMeans(n_clusters)
# km.fit(finalImg)

# df = pd.DataFrame(finalImg, columns=['R','G', 'B'])
# df['label'] = km.labels_

# colors = plt.cm.Spectral(np.linspace(0, 1, len(df.label.unique())))

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# for color, label in zip(colors, df.label.unique()):
    
#     tempdf = df[df.label == label]
#     ax.scatter(tempdf.R, tempdf.G, tempdf.B, c = color)
# #     plt.scatter(tempdf.x, tempdf.y, c=color)
    
# ax.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:, 1],km.cluster_centers_[:,2], c='r', s=500, alpha=0.7, )
# plt.grid(True)
# ax.set_xlabel('Red')
# ax.set_ylabel('Green')
# ax.set_zlabel('Blue')
# plt.show()

# combined code

In [ ]:
import os
from shutil import copyfile

folderPath = r"C:\Users\sohil\AnacondaProjects\JobTask\example_output\\"
threshold = 1.8

# # Create directory
# dirName = 'NoisyData'
# try:
#     # Create target Directory
#     os.mkdir(dirName)
#     print("Directory " , dirName ,  " Created ") 
# except FileExistsError:
#     print("Directory " , dirName ,  " already exists")
    
# for i in range(1,1965):
i = 1
imgPath = folderPath + "extract_field%d.jpg" % i
img = readImg(imgPath)
filtered = filterImg(img)
zscore = zScore(filtered)
filtered = removeOutliers(filtered, zscore, threshold)
K = findOptimalK(filtered)

i = 2
imgPath = folderPath + "extract_field%d.jpg" % i
img = readImg(imgPath)
filtered = filterImg(img)
zscore = zScore(filtered)
filtered = removeOutliers(filtered, zscore, threshold)
K = findOptimalK(filtered)

#     if K > 2:
#         print("Noisy Data")
#         src = imgPath
#         dst = "%i.png"%i
#         copyfile(src, dst)

Optimal clusters: 3
